In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import wxconv
import glob
from wxconv import WXC
import codecs
import os,subprocess

In [2]:
pwd()


'C:\\Users\\aparashar'

In [3]:
%cd E:\\META\\ltrc\\ace-0.9.24

E:\META\ltrc\ace-0.9.24


In [2]:
def get_pos(pos,meaning):
    if "adj." in meaning:
        return "adj"
    elif "to " in meaning:
        return "verb"
    elif "adv. " in meaning:
        return "adv"
    elif pos != "":
        return pos
    else:
        return "noun"

In [3]:
# filtering words and meaning from the html pages
def get_word_meaning(results,d,ind):
    for i in results[1:]:
        try:
            hw = i.find("hw")
            pos = ""
            #print(hw)
            if hw:
                if "adj." in hw.next_sibling:
                    pos = "adj"
                elif "adv. " in hw.next_sibling:
                    pos = "adv"
            
            if hw and i.find("def"):

                defs = i.find_all("def")

                for j in defs[1:]:
                    t = j.previous_sibling
                    if t[0].isdigit():
                        t = t[3:]
                    colon_split = t.split(";")
                    for _ in colon_split:
                        comma_splt = _.split(",")
                        for k in comma_splt:
                            d["root"].append(hw.text.split(" ")[0])
                            d["word"].append(hw.text.split(" ")[0])
                            d["meaning"].append(k.strip())
                            d["pos"].append(get_pos(pos,k.strip(' ')))
                t = defs[-1].contents[0]
                if t[0].isdigit():
                        t = t[3:]
                colon_split = t.split(";")
                for _ in colon_split:
                    comma_splt = _.split(",")
                    for k in comma_splt:
                        d["root"].append(hw.text.split(" ")[0])
                        d["word"].append(hw.text.split(" ")[0])
                        d["meaning"].append(k.strip())
                        d["pos"].append(get_pos(pos,k.strip(' ')))

            elif hw and (i.find("def")==None):
                if i.find("i"):
                    i_s = i.find_all("i")
                    t = i_s[-1].next_sibling
                else:
                    t = hw.next_sibling
                    
                colon_split = t.split(";")
                for _ in colon_split:
                    comma_splt = _.split(",")
                    for k in comma_splt:
                        d["root"].append(hw.text.split(" ")[0])
                        d["word"].append(hw.text.split(" ")[0])
                        d["meaning"].append(k.strip())
                        d["pos"].append(get_pos(pos,k.strip(' ')))
            
            shws = i.find_all("shw")
            for j in shws:
                pos = ""
                if "adj." in j.next_sibling:
                    pos = "adj"
                elif "adv. " in j.next_sibling:
                    pos = "adv"
                if "~" in j.contents[0]:
                    devas = j.find_all("deva")
                    for k in devas:
                        word = hw.text.split(" ")[0] +" "+ k.text
                        #print([word,j.next_sibling])
                        colon_split = j.next_sibling.split(";")
                        for _ in colon_split:
                            comma_splt = _.split(",")
                            for k in comma_splt:
                                d["root"].append(hw.text.split(" ")[0])
                                d["word"].append(word)
                                d["meaning"].append(k.strip())
                                d["pos"].append(get_pos(pos,k.strip(' ')))

                elif "˚" in j.contents[0]:
                    deva = j.find("deva")
                    og = hw.text.split(" ")[0]
                    word = d["word"][-1] + "˚"+ deva.text
                    if "[" in j.next_sibling:
                        while  "]" not in j.next_sibling:
                            j = j.next_sibling
                    colon_split = j.next_sibling.split(";")
                    for _ in colon_split:
                        comma_splt = _.split(",")
                        for k in comma_splt:
                            d["root"].append(og)
                            d["word"].append(word)
                            d["meaning"].append(k.strip())
                            d["pos"].append(get_pos(pos,k.strip(' ')))
                else:
                    devas = j.find_all("deva")
                    for k in devas:
                        word = hw.text.split(" ")[0] + " "+ k.text
                        #word = k.text
                        #print([word,j.next_sibling])
                        if "[" in j.next_sibling:
                            while  "]" not in j.next_sibling:
                                j = j.next_sibling
                        #print(j.next_sibling)
                        colon_split = j.next_sibling.split(";")
                        for _ in colon_split:
                            comma_splt = _.split(",")
                            for k in comma_splt:
                                d["root"].append(hw.text.split(" ")[0])
                                d["word"].append(word)
                                d["meaning"].append(k.strip())
                                d["pos"].append(get_pos(pos,k.strip(' ')))
        except:
            # returning which words on which pages threw exception
            print([ind,hw])
            continue
   

#print(d)

In [4]:
d = {"word":[],"meaning":[],"pos":[],"root":[]}
pindex = 1
# total 1083 pages
for i in range(1083):
    URL = "https://dsal.uchicago.edu/cgi-bin/app/mcgregor_query.py?page=" + str(i)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="results_display")
    results  = results.find_all("div")
    while soup.find("sp"):
        soup.sp.extract()
    get_word_meaning(results,d,i)

[50, <hw><deva>अभी</deva> <tran>abhī</tran></hw>]
[61, <hw><deva>अवतार</deva> <tran>avă-tār</tran></hw>]
[95, <hw><deva>आवाज़</deva> <tran>āvāz</tran></hw>]
[98, <hw><deva>आसन्न</deva> <tran>ā-sann</tran></hw>]
[99, <hw><deva>आह</deva> <tran>āh</tran></hw>]
[105, <hw><deva>इनाम</deva> <tran>inām</tran></hw>]
[107, <hw><deva>इलायची</deva> <tran>ilāycī</tran></hw>]
[110, <hw><deva>ईमान</deva> <tran>īmān</tran></hw>]
[112, <hw><deva>उँगली</deva> <tran>uṁglī</tran></hw>]
[114, <hw><deva>उचाट</deva> <tran>ucāṭ</tran></hw>]
[123, <hw><deva>उदर</deva> <tran>udar</tran></hw>]
[131, <hw><deva>उम्मीद</deva> <tran>ummīd</tran></hw>]
[136, <hw><deva>ऊँचा</deva> <tran>ūṁcā</tran></hw>]
[136, <hw><deva>ऊँट</deva> <tran>ūṁṭ</tran></hw>]
[142, <hw><deva>एतबार</deva> <tran>etbār</tran></hw>]
[147, <hw><deva>ओर</deva> <tran>or</tran></hw>]
[151, <hw><deva>औसान</deva> <tran>ausān</tran></hw>]
[162, <hw><deva>क़तरा</deva> <tran>qatrā</tran></hw>]
[166, <hw><deva>कफ़</deva> <tran>kaf</tran></hw>]
[172, <hw><d

In [6]:
df =pd.DataFrame.from_dict(d)

In [7]:
# dropping improper meaning rows
l = ['[S.]',']','.','˚]']
df = df[df['meaning'].str.strip().astype(bool)]
indexNames = df[df['meaning'].isin(l)].index
df.drop(indexNames,inplace=True)
df.reset_index(drop = True, inplace = True)
#df[300:320]

In [102]:
#df.to_csv('testdf.txt',sep ='\t')

In [13]:
df.loc[df.meaning.apply(lambda row: "= next" in row),"meaning"] = df["meaning"].shift(-1)

In [14]:
df[df.meaning.apply(lambda row: "= next" in row)]

,word,meaning,pos,root


In [15]:
# removing unwanted characters from the text
unw = ['. —','m. ','adv. ','id.','adj. ','hw.','f. ',' (=','&c.','= ~','(= ~',')','Brbh. =','v.t.','Brbh.','Av.','v.i.','inv.','U.','? ← Fr.]','&','.','[',']','-','=',' –']
def clean_set(m):
    ret = m
    ret = re.sub("[\(\[].*?[\)\]]", "", m)
    ret = ret.split('(')[0]
    for i in unw:
        ret = ret.replace(i,'')
    
    return ret.strip()
    

In [18]:
# last check for 'verb' pos
def handle_1(row):
    to = ['to ', 'To ']
    tobe = ['to be ','To be ']
    if row['meaning'][:3] in to and row['meaning'][:6] not in tobe:
        return 'verb'
    else:
        return row['pos']
# removing 'ना' suffix if the pos is a verb
def handle_2(row):
    row["word"] = re.sub("[\(\[].*?[\)\]]", "", row["word"])
    row['word'] = row['word'].strip(' ')
    if row['pos'] == 'verb' and row['word'][-2:] == 'ना':
        l = len(row['word'])
        return row['word'][:l-2]
    else:
        return row['word']

# removing articles from the beginning of meaning
def handle_3(row):
    ret = row['meaning']
    space_splt = ret.split(' ')
    arts = ['a','the','an']
    if space_splt[0] in arts:
        ret = ' '.join(space_splt[1:])
    if space_splt[0] == 'to' and row['pos'] == 'verb':
        ret = ' '.join(space_splt[1:])
    return ret


def get_sentence(word,pos):
    if pos == 'verb' :
        return 'Please {}.'.format(word)
    elif pos == 'adj' :
        return 'This is {}.'.format(word)
    elif pos == 'noun' :
        return 'This is the {}.'.format(word)
    if pos == 'adv': 
        return 'He wants to do it {}.'.format(word)
    else:
        return word

In [19]:
df['meaning'] = df.meaning.apply(lambda row: clean_set(row))
df['pos'] = df.apply(lambda row: handle_1(row),axis=1)
df['word'] = df.apply(lambda row: handle_2(row),axis=1)
df['meaning'] = df.apply(lambda row: handle_3(row),axis=1)
#df.drop(df.index[df['word'] == None], inplace=True)

In [20]:
df

,word,meaning,pos,root
0,अ,first vowel of the Devanāgarī syllabary,noun,अ
1,अ अकार,sound /ə/,noun,अ
2,अ अकार,letter,noun,अ
3,अ अकार˚आदि क्रम,alphabetical order,noun,अ
4,अंक,number,noun,अंक
...,...,...,...,...
149565,हौसला हौसलामंद,aspiring,adj,हौसला
149566,हौसला हौसलामंद,ambitious,noun,हौसला
149567,हौसला हौसलामंद,bold,noun,हौसला
149568,ह्याँ,,adv,ह्याँ


In [22]:
def handle_4(row):
    ret = row['word']
    ret = ret.strip(' ')
    return ret.replace(' ','+')
def handle_5(row):
    pos = row['pos']
    og = row['root']
    word = row['word']
    split_plus = word.split('+')
    if '+' in word:
        #if split_plus[0] == og and pos != 'verb':
            #return ('').join(split_plus[1:])
        if split_plus[0] == og and (split_plus[1] == og or og in split_plus[1]):
            print([word])
            return ('').join(split_plus[1:])
        else:
            return word
    else:
        return word
            

In [28]:
# filtering single worded meanings from the df dataframe to sdf dataframe
#sdf = df[df.meaning.apply(lambda row: row.find(' ') == -1)]
sdf = df
sdf['word'] = sdf.apply(lambda row:handle_4(row),axis =1)
sdf['word'] = sdf.apply(lambda row:handle_5(row),axis =1)
sdf['word'] = sdf.word.apply(lambda x: x.strip('-'))
sdf = sdf[sdf.meaning != '']
#sdf[sdf['pos']=='verb']

In [29]:
sdf[11000:11020]

,word,meaning,pos,root
11831,आतुरता,impatience,noun,आतुरता
11834,आत्म,in comp see sv,noun,आत्म-
11835,आत्मक,formant having the nature or character of,adj,-आत्मक
11836,आत्मनेपद,middle voice,noun,आत्मनेपद
11837,आत्मा,soul,noun,आत्मा
11838,आत्मा,spirit,noun,आत्मा
11839,आत्मा,self,noun,आत्मा
11840,आत्मा,individual,noun,आत्मा
11841,आत्मा,mind,noun,आत्मा
11842,आत्मा,heart,noun,आत्मा


In [30]:
def isCompound(row):
    word = row['word']
    meaning = row['meaning']
    if '+' in word or '-' in word or ' ' in meaning:
        return 1
    return 0
sdf['comp'] = sdf.apply(lambda x:isCompound(x),axis=1)
csdf = sdf[sdf.comp ==1]

/var/folders/hz/2sszhln51pvb6wvkp4z5vdcr0000gn/T/ipykernel_56963/3087208071.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['comp'] = sdf.apply(lambda x:isCompound(x),axis=1)


In [32]:
csdf.reset_index(inplace= True)

In [33]:
csdf.drop(['root','comp'],inplace=True,axis=1)

/var/folders/hz/2sszhln51pvb6wvkp4z5vdcr0000gn/T/ipykernel_56963/1135257010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csdf.drop(['root','comp'],inplace=True,axis=1)


In [35]:
csdf.drop(['index'],inplace=True,axis=1)

/var/folders/hz/2sszhln51pvb6wvkp4z5vdcr0000gn/T/ipykernel_56963/1871075807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csdf.drop(['index'],inplace=True,axis=1)


In [37]:
csdf.to_csv('mcgregor_hindi-eng-compound-v1.txt',sep='\t')
csdf.to_csv('mcgregor_hindi-eng-compound-v1.csv',sep=',')

In [14]:
def getLines(file,splitby):
    with codecs.open(file, 'r', encoding="utf-8") as filer:
                    data = filer.read()
                    data = re.sub('\t+', '\t', data)
    lines = data.split(splitby)
    return lines

In [5]:
con = WXC(order='utf2wx', lang='hin')
con.convert('अभिख्या')

'aBiKyA'

In [168]:
mrs ={'conc_label_wx':[],'conc_label_utf':[],'conc_in_eng':[],'mrs_conc':[]}
con = WXC(order='wx2utf', lang='hin')
for line in getLines('H_concept-to-mrs-rels.dat','\n'):
    internal_lines = line.split('\t')
    if len(internal_lines)>=4:
        mrs['conc_label_wx'].append(internal_lines[1])
        mrs['conc_label_utf'].append(con.convert(internal_lines[1]).replace('+',' '))
        mrs['conc_in_eng'].append(internal_lines[2])
        mrs['mrs_conc'].append(internal_lines[3])
    else:
        print(internal_lines[2])
        mrs['conc_label_wx'].append(internal_lines[1])
        mrs['conc_label_utf'].append(con.convert(internal_lines[1]).replace('+',' '))
        if len(internal_lines[2])>20:
            mrs['conc_in_eng'].append(internal_lines[2].split(' ')[0])
            mrs['mrs_conc'].append(internal_lines[2].split(' ')[1])
        else:
            mrs['conc_in_eng'].append(internal_lines[2][0:7])
            mrs['mrs_conc'].append(internal_lines[2][8:])
mrs_df = pd.DataFrame.from_dict(mrs)
mrs_df

foundation+stone_1 _foundation_n_1+_stone_n_1)
dutch_1_dutch_a_1_1)
shrug_3_shrug_n_1_1)


,conc_label_wx,conc_label_utf,conc_in_eng,mrs_conc
0,A_1,आ_1,come_1,_come_v_1)
1,Aba_1,आब_1,lustre_1,_lustre/NN_u_unknown)
2,ABA_1,आभा_1,lustre_2,_lustre/NN_u_unknown)
3,aba_1,अब_1,now_3,def_implicit_q)
4,aba_1,अब_1,now_3,loc_nonsp)
...,...,...,...,...
54324,bAriSa_1,बारिश_1,rain_4,_rain_v_1)
54325,kaha_3,कह_3,somewhere_1,_place_n_of)
54326,aBI+BI_1,अभी भी_1,still_16,_still_a_1)
54327,jArI+raKa_1,जारी रख_1,continue_2,_continue_v_2)


In [152]:
def get_in_og_mrs(row):
    word = row['word']
    meaning = row['meaning']
    c1 = mrs_df.conc_label_utf.apply(lambda r : r.split('_')[0] == word)
    c2 = mrs_df.conc_in_eng.apply(lambda r: r.split('_')[0]==meaning)
    
    cross = mrs_df[c1 & c2]
    if cross.empty == True:
        return 0
    else:
        return 1

In [153]:
sdf['in_og_mrs'] = sdf.apply(lambda row: get_in_og_mrs(row),axis=1)


In [154]:
sdf_r = sdf[sdf.in_og_mrs.apply(lambda x: x == 0)]
sdf_r

,index,word,meaning,pos,root,in_og_mrs
2,4,अकार,letter,noun,अ,0
7,10,अंक,spot,noun,अंक,0
8,11,अंक,line,noun,अंक,0
9,12,अंक,stamp,noun,अंक,0
10,13,अंक,brand,noun,अंक,0
...,...,...,...,...,...,...
149555,200892,हौसला,resolve,noun,हौसला,0
149564,200903,हौसलाअफ़ज़ाई,encouragement,noun,हौसला,0
149565,200905,हौसलामंद,aspiring,adj,हौसला,0
149566,200906,हौसलामंद,ambitious,noun,हौसला,0


In [76]:
sdf_r.to_csv('sdf-1.txt',sep='\t')

In [190]:
mrs_word_cnt = {}
mrs_mean_cnt = {}
def count_mrs_word(row):
    word = row['conc_label_utf'].split('_')[0]
    word.replace(' ','+')
    if word in mrs_word_cnt:
        mrs_word_cnt[word] +=1
        
    else:
        mrs_word_cnt[word] =1
def count_mrs_mean(row):
    word = row['conc_in_eng'].split('_')[0]
    if word in mrs_mean_cnt:
        mrs_mean_cnt[word] +=1
        
    else:
        mrs_mean_cnt[word] =1
#mrs_df

In [191]:
mrs_df['conc_label_utf'] = mrs_df.conc_label_utf.apply(lambda x: x.replace(' ','+') )
mrs_df.apply(lambda row:count_mrs_word(row),axis =1)

mrs_word_cnt

{'आ': 1,
 'आब': 3,
 'आभा': 4,
 'अब': 4,
 'अबाबील': 1,
 'अभागा': 3,
 'अभागिन': 1,
 'अभाग्य': 1,
 'आबहवा': 1,
 'आबकारी': 1,
 'अभक्ष्य': 2,
 'आभामण्डल': 1,
 'अभंग': 2,
 'आबंटन': 1,
 'आबनूस': 1,
 'आबपाशी': 3,
 'आभार': 4,
 'अबरक': 1,
 'अभरक': 1,
 'आभरण': 2,
 'आभारी': 3,
 'आबरू': 2,
 'आभास': 6,
 'आभासी': 3,
 'अभाव': 5,
 'अभावित': 1,
 'आबताब': 3,
 'अब+तक': 1,
 'अबाध': 2,
 'आबाद': 3,
 'आबादी': 2,
 'अबाधित': 3,
 'अभद्र': 3,
 'अभद्र+व्यवहार': 1,
 'आबद्ध': 1,
 'अबद्ध': 2,
 'अभय': 1,
 'अब्बा': 1,
 'आबेहयात': 1,
 'अभेद': 2,
 'अभेद्य': 4,
 'अभेद्यता': 1,
 'अभी': 3,
 'अभिभव': 3,
 'अभिभावक': 1,
 'अभिभूत': 3,
 'अभिचार': 2,
 'अभिचाररक': 1,
 'अभिचाररक्रिया': 2,
 'अभिचारी': 2,
 'अभिग्रहण': 1,
 'अभिहित': 3,
 'अभिजात': 1,
 'अभिजातवर्ग': 2,
 'अभिजात+वर्ग': 1,
 'अभिजातवाद': 2,
 'अभिजातवादी': 1,
 'अभिजाततंत्र': 1,
 'आभिजात्य': 2,
 'अभिजात्य': 2,
 'अभिज्ञान': 4,
 'अभिज्ञता': 2,
 'अभिज्ञात': 1,
 'अभिकल्प': 1,
 'अभिकर्ता': 1,
 'अभिकथन': 1,
 'अभिक्रियारिया': 1,
 'अभिलक्षण': 1,
 'अभिलम्ब': 1,
 'अभिलाषा': 5,
 'अभिला

In [123]:

# adding count at the end of the words and meanings

def count_word(row):
    word = row['word']
    if word in mrs_word_cnt.keys():
        mrs_word_cnt[word] +=1
        return word+'_'+ str(mrs_word_cnt[word])
    else:
        mrs_word_cnt[word] =1
        return word+'_'+ str(mrs_word_cnt[word])
def count_meaning(row):
    meaning = row['meaning']
    if meaning in mrs_mean_cnt.keys():
        mrs_mean_cnt[meaning] +=1
        return meaning+'_'+ str(mrs_mean_cnt[meaning])
    else:
        mrs_mean_cnt[meaning] =1
        return meaning+'_'+ str(mrs_mean_cnt[meaning])
        

In [158]:
sdf_r['word'] = sdf_r.apply(lambda row:count_word(row),axis =1)
sdf_r['meaning'] = sdf_r.apply(lambda row:count_meaning(row),axis =1)

sdf_r

C:\Users\aparashar\AppData\Local\Temp\ipykernel_5744\1859008332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf_r['word'] = sdf_r.apply(lambda row:count_word(row),axis =1)
C:\Users\aparashar\AppData\Local\Temp\ipykernel_5744\1859008332.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf_r['meaning'] = sdf_r.apply(lambda row:count_meaning(row),axis =1)


,index,word,meaning,pos,root,in_og_mrs
2,4,अकार_1,letter_1,noun,अ,0
7,10,अंक_12,spot_1,noun,अंक,0
8,11,अंक_13,line_1,noun,अंक,0
9,12,अंक_14,stamp_1,noun,अंक,0
10,13,अंक_15,brand_1,noun,अंक,0
...,...,...,...,...,...,...
149555,200892,हौसला_6,resolve_16,noun,हौसला,0
149564,200903,हौसलाअफ़ज़ाई_1,encouragement_12,noun,हौसला,0
149565,200905,हौसलामंद_1,aspiring_2,adj,हौसला,0
149566,200906,हौसलामंद_2,ambitious_3,noun,हौसला,0


In [409]:
sdf_r

NameError: name 'sdf_r' is not defined

In [160]:
#sdf_r = sdf_r.reset_index()
sdf_r = sdf_r.drop(columns=['index'])
sdf_r.to_csv("1-1083_v5.txt",sep="\t")

In [74]:
sdf_r = sdf_r.reset_index()
sdf_r = sdf_r.drop(columns=['index'])
sdf_r.to_csv("1-1083_v3.txt",sep="\t")
sdf_r

,word,meaning,pos
0,अकार_1,letter_1,noun
1,अंक_12,spot_1,noun
2,अंक_13,line_1,noun
3,अंक_14,stamp_1,noun
4,अंक_15,brand_1,noun
...,...,...,...
70778,ह्रस्वता_1,shortness_2,noun
70779,हृदिनी_1,lightning_10,noun
70780,ह्रास_1,declining_3,noun
70781,ह्रासोन्मुख_1,declining_4,noun


In [192]:
final_df = pd.read_csv('1-1083_v3_dict.txt',sep='\t')

In [264]:
mcg_df_v5 = pd.read_csv('1-1083_v5.txt',sep='\t')
mcg_df_v5 = mcg_df_v5.drop(['root','in_og_mrs'],axis =1)
mcg_df_v5.to_csv('mcgregor_wordnet_v5.txt',sep='\t')

In [193]:
import wxconv
import numpy as np
from wxconv import WXC

In [194]:
final_df

,Hin_Label,Eng_Label,POS,MRS_Concept,MRS_Feature_Values
0,word,meaning,pos,unknown,LBL: h* ARG0: e* ARG: x*
1,word,meaning,pos,udef_q,LBL: h* ARG0: x* RSTR: h* BODY: h*
2,word,meaning,pos,_meaning_n_1,LBL: h* ARG0: x*
3,अकार_1,letter_1,noun,_letter_n_of,LBL: h* ARG0: x* ARG1: i*
4,अंक_13,line_1,noun,_line_n_of,LBL: h* ARG0: x*
...,...,...,...,...,...
67882,हौसला_6,resolve_16,noun,_resolve_n_1,LBL: h* ARG0: x*
67883,हौसलाअफ़ज़ाई_1,encouragement_12,noun,_encouragement_n_of,LBL: h* ARG0: x* ARG1: i*
67884,हौसलामंद_1,aspiring_2,adj,_aspiring_a_1,LBL: h* ARG0: e* ARG1: x*
67885,"Oops!<class 'IndexError'>occurred.ambitious_3,...",NaN,NaN,NaN,NaN


In [201]:
con = WXC(order='utf2wx')
def drop_digit(row):
    word = row.split('_')[0]
    if word.isdigit():
        return 1
    else:
        return 0

In [203]:
final_df['isdigit'] = final_df.Eng_Label.apply(lambda x : drop_digit(x))
final_df = final_df[final_df['isdigit'] ==0]
final_df = final_df[final_df.Hin_Label.apply(lambda x :'Oops' not in x )]
final_df = final_df[final_df.Hin_Label.apply(lambda x :'word' not in x )]

In [318]:
final_df

,conc_label_utf,conc_in_eng,POS,mrs_conc,MRS_Feature_Values,isdigit
3,अकार_1,letter_1,noun,_letter_n_of,LBL: h* ARG0: x* ARG1: i*,0
4,अंक_12,line_1,noun,_line_n_of,LBL: h* ARG0: x*,0
5,अंक_13,stamp_1,noun,_stamp_n_1,LBL: h* ARG0: x*,0
6,अंक_14,pricemark_1,noun,_pricemark/NN_u_unknown,LBL: h* ARG0: x*,0
7,अंक_15,syllable_1,noun,_syllable_n_1,LBL: h* ARG0: x*,0
...,...,...,...,...,...,...
67880,हौसला_4,courage_14,noun,_courage_n_1,LBL: h* ARG0: x*,0
67881,हौसला_5,energy_14,noun,_energy_n_1,LBL: h* ARG0: x*,0
67882,हौसला_6,resolve_15,noun,_resolve_n_1,LBL: h* ARG0: x*,0
67883,हौसलाअफ़ज़ाई_1,encouragement_10,noun,_encouragement_n_of,LBL: h* ARG0: x* ARG1: i*,0


In [205]:
def recount_word(row):
    word = row['Hin_Label'].split('_')[0]
    if word in mrs_word_cnt.keys():
        mrs_word_cnt[word] +=1
        return word+'_'+ str(mrs_word_cnt[word])
    else:
        mrs_word_cnt[word] =1
        return word+'_'+ str(mrs_word_cnt[word])
def recount_meaning(row):
    meaning = row['Eng_Label'].split('_')[0]
    if meaning in mrs_mean_cnt.keys():
        mrs_mean_cnt[meaning] +=1
        return meaning+'_'+ str(mrs_mean_cnt[meaning])
    else:
        mrs_mean_cnt[meaning] =1
        return meaning+'_'+ str(mrs_mean_cnt[meaning])
        

In [218]:
mrs_word_cnt['अंक']

17

In [317]:
final_df.columns.tolist()

['conc_label_utf',
 'conc_in_eng',
 'POS',
 'mrs_conc',
 'MRS_Feature_Values',
 'isdigit']

In [398]:
f = {}
def getconc_feat_dict(a,b):
    if a not in f.keys():
        f[a] = [b.strip(' ')+')']
    if a in f.keys() and b.strip(' ') + ')' not in f[a]:
        f[a].append(b.strip(' ')+')')



In [399]:

final_df.apply(lambda row:getconc_feat_dict(row['mrs_conc'],row['MRS_Feature_Values']),axis=1)
len(f)
f
mcg_cft ={'MRS_Concept':[],'MRS_Feature_Values':[]}
for i in f.keys():
    
    for j in f[i]:
        mcg_cft['MRS_Concept'].append(i)
        mcg_cft['MRS_Feature_Values'].append(j)
mcg_cft_df = pd.DataFrame.from_dict(mcg_cft)
mcg_cft_df.to_csv('mcgregor_concept_feature_rels.txt',sep='\t')


In [400]:
def get_lab(row):
    return '(MRS_concept-label-feature_values'

In [401]:
mcg_cft_df['label'] = mcg_cft_df.apply(lambda x: get_lab(x))
mcg_cft_df = mcg_cft_df.assign(label='(MRS_concept-label-feature_values')

mcg_cft_df['MRS_Feature_Values'] = mcg_cft_df.MRS_Feature_Values.apply(lambda x : x)
mcg_cft_df

,MRS_Concept,MRS_Feature_Values,label
0,_letter_n_of,LBL: h* ARG0: x* ARG1: i*),(MRS_concept-label-feature_values
1,_line_n_of,LBL: h* ARG0: x*),(MRS_concept-label-feature_values
2,_stamp_n_1,LBL: h* ARG0: x*),(MRS_concept-label-feature_values
3,_pricemark/NN_u_unknown,LBL: h* ARG0: x*),(MRS_concept-label-feature_values
4,_syllable_n_1,LBL: h* ARG0: x*),(MRS_concept-label-feature_values
...,...,...,...
15145,_bet_v_1,LBL: h* ARG0: e* ARG1: x* ARG2: p*),(MRS_concept-label-feature_values
15146,_priestess/NN_u_unknown,LBL: h* ARG0: x*),(MRS_concept-label-feature_values
15147,_feasability/NN_u_unknown,LBL: h* ARG0: x*),(MRS_concept-label-feature_values
15148,_bonfire_n_1,LBL: h* ARG0: x*),(MRS_concept-label-feature_values


In [402]:
cols = mcg_cft_df.columns.tolist()
cols = ['label','MRS_Concept','MRS_Feature_Values']
mcg_cft_df = mcg_cft_df[cols]

In [403]:
mcg_cft_df.to_csv('mcgregor_concept_feature_rels.dat',sep=' ',index=False)

In [357]:

for line in getLines('mrs-rel-features.dat','\n'):
    internal_lines = line.split(' ')
    if len(internal_lines)>=2:
        a = internal_lines[1]
        b = ' '.join(internal_lines[2:]).replace(')','')
        if a not in f.keys():
            #print('hajaaja')
            f[a] = [b]
        if a in f.keys() and b not in f[a]:
            #print([a,b,f[a]])
            f[a].append(b)
        
       
    

len(f)

22634

In [427]:
cft ={'MRS_Concept':[],'MRS_Feature_Values':[]}
for i in f.keys():
    
    for j in f[i]:
        cft['MRS_Concept'].append(i)
        cft['MRS_Feature_Values'].append(j)
cft_df = pd.DataFrame.from_dict(cft)
cft_df[cft_df.MRS_Concept == '_lustre/NN_u_unknown']

,MRS_Concept,MRS_Feature_Values
3824,_lustre/NN_u_unknown,LBL: h* ARG0: x*)


In [404]:
cft_df = cft_df.assign(label='(MRS_concept-label-feature_values')

cft_df['MRS_Feature_Values'] = cft_df.MRS_Feature_Values.apply(lambda x : x+')')
cft_df

,MRS_Concept,MRS_Feature_Values,label
0,_letter_n_of,LBL: h* ARG0: x* ARG1: i*),(MRS_concept-label-feature_values
1,_line_n_of,LBL: h* ARG0: x*),(MRS_concept-label-feature_values
2,_line_n_of,LBL: h* ARG0: e* ARG1: x* ARG2: x*),(MRS_concept-label-feature_values
3,_stamp_n_1,LBL: h* ARG0: x*),(MRS_concept-label-feature_values
4,_pricemark/NN_u_unknown,LBL: h* ARG0: x*),(MRS_concept-label-feature_values
...,...,...,...
26740,_zoologist/NN_u_unknown,LBL: h* ARG0: x*),(MRS_concept-label-feature_values
26741,_zoom_n_1,LBL: h* ARG0: x*),(MRS_concept-label-feature_values
26742,_zoom_v_1,LBL: h* ARG0: e* ARG1: x*),(MRS_concept-label-feature_values
26743,_playground_n_1,LBL: h* ARG0: x*),(MRS_concept-label-feature_values


In [405]:
cols = cft_df.columns.tolist()
cols = ['label','MRS_Concept','MRS_Feature_Values']
cft_df = cft_df[cols]

In [406]:
cft_df.to_csv('aggregate_concept_feature_rels.dat',sep=' ',index=False)

In [207]:
final_df['Hin_Label'] = final_df.Hin_Label.apply(lambda x:x.replace('~',''))
final_df['Hin_Label'] = final_df.Hin_Label.apply(lambda x:x.replace('++','+'))
final_df['Hin_Label'] = final_df.apply(lambda row:recount_word(row),axis =1)
final_df['Eng_Label'] = final_df.apply(lambda row:recount_meaning(row),axis =1)


final_df

,Hin_Label,Eng_Label,POS,MRS_Concept,MRS_Feature_Values,isdigit
3,अकार_1,letter_1,noun,_letter_n_of,LBL: h* ARG0: x* ARG1: i*,0
4,अंक_12,line_1,noun,_line_n_of,LBL: h* ARG0: x*,0
5,अंक_13,stamp_1,noun,_stamp_n_1,LBL: h* ARG0: x*,0
6,अंक_14,pricemark_1,noun,_pricemark/NN_u_unknown,LBL: h* ARG0: x*,0
7,अंक_15,syllable_1,noun,_syllable_n_1,LBL: h* ARG0: x*,0
...,...,...,...,...,...,...
67880,हौसला_4,courage_14,noun,_courage_n_1,LBL: h* ARG0: x*,0
67881,हौसला_5,energy_14,noun,_energy_n_1,LBL: h* ARG0: x*,0
67882,हौसला_6,resolve_15,noun,_resolve_n_1,LBL: h* ARG0: x*,0
67883,हौसलाअफ़ज़ाई_1,encouragement_10,noun,_encouragement_n_of,LBL: h* ARG0: x* ARG1: i*,0


In [209]:
final_df
final_df.rename(columns={'Hin_Label': 'conc_label_utf', 'Eng_Label': 'conc_in_eng','MRS_Concept':'mrs_conc'}, inplace=True)
f_df = final_df.drop(['POS','MRS_Feature_Values'],axis=1)
f_df['conc_label_wx'] = f_df.conc_label_utf.apply(lambda x : con.convert(x))



In [210]:
cols = f_df.columns.tolist()
cols = ['conc_label_wx','conc_label_utf','conc_in_eng','mrs_conc']
f_df = f_df[cols]

In [211]:
f_df [455:465]

,conc_label_wx,conc_label_utf,conc_in_eng,mrs_conc
537,akRoni_1,अक्षोनि_1,army_1,_army_n_of
538,aksa_3,अक्स_3,reverse_1,_reverse_n_of
543,aKaMda_4,अखंड_4,unbroken_1,_unbroken_a_1
544,aKaMda_5,अखंड_5,unlimited_1,_unlimited_a_1
545,aKaMda_6,अखंड_6,indestructible_1,_indestructible/JJ_u_unknown
546,aKaMdawA_4,अखंडता_4,entirety_1,_entirety_n_1
547,aKaMdanIya_6,अखंडनीय_6,incontestable_1,_incontestable/JJ_u_unknown
548,aKaja-axAvawa_1,अखज-अदावत_1,hostility_1,_hostility_n_1
549,aKaja-axAvawa_2,अखज-अदावत_2,illwill_1,_illwill/NN_u_unknown
551,aKadZAnA_1,अखड़ाना_1,rude_1,_user_n_of


In [212]:
mrs_df[mrs_df.conc_label_utf.apply(lambda x:'+' in x)]

,conc_label_wx,conc_label_utf,conc_in_eng,mrs_conc
61,aba+waka_1,अब+तक_1,heretofore_1,_heretofore_a_1)
75,aBaxra+vyavahAra_1,अभद्र+व्यवहार_1,lip_7,_lip_n_1)
112,aBijAwa+varga_1,अभिजात+वर्ग_1,elite_1,_elite_n_1)
149,aBileKana+kara_1,अभिलेखन+कर_1,tape_3,_tape_v_1)
176,aBinaya+kara_1,अभिनय+कर_1,act_5,_act_v_1)
...,...,...,...,...
54280,yuxXa+kOSala+sambanXI_1,युद्ध+कौशल+सम्बन्धी_1,strategic_2,_strategic_a_1)
54296,yuxXa+saMbaMXI_1,युद्ध+संबंधी_1,martial_5,_martial_a_1)
54326,aBI+BI_1,अभी+भी_1,still_16,_still_a_1)
54327,jArI+raKa_1,जारी+रख_1,continue_2,_continue_v_2)


In [213]:
concat_df = pd.concat([ mrs_df,f_df])
concat_df.reset_index()

,index,conc_label_wx,conc_label_utf,conc_in_eng,mrs_conc
0,0,A_1,आ_1,come_1,_come_v_1)
1,1,Aba_1,आब_1,lustre_1,_lustre/NN_u_unknown)
2,2,ABA_1,आभा_1,lustre_2,_lustre/NN_u_unknown)
3,3,aba_1,अब_1,now_3,def_implicit_q)
4,4,aba_1,अब_1,now_3,loc_nonsp)
...,...,...,...,...,...
112560,67880,hOsalA_4,हौसला_4,courage_14,_courage_n_1
112561,67881,hOsalA_5,हौसला_5,energy_14,_energy_n_1
112562,67882,hOsalA_6,हौसला_6,resolve_15,_resolve_n_1
112563,67883,hOsalAaPZajZAI_1,हौसलाअफ़ज़ाई_1,encouragement_10,_encouragement_n_of


In [231]:
#sAmaMwa
concat_df['mrs_conc']=concat_df.mrs_conc.apply(lambda x: x.replace(')',''))

In [214]:
def get_in_og_mrs2(row):
    word = row['conc_label_utf'].split('_')[0]
    meaning = row['conc_in_eng'].split('_')[0]
    c1 = concat_df.conc_label_utf.apply(lambda r : r.split('_')[0] == word)
    c2 = concat_df.conc_in_eng.apply(lambda r: r.split('_')[0]==meaning)
    
    cross = concat_df[c1 & c2]
    if cross.empty == True:
        return 0
    else:
        return 1

In [215]:
augmrs_df['in_og'] = augmrs_df.apply(lambda row: get_in_og_mrs2(row),axis=1)

In [216]:
#augmrs_df = pd.read_excel('augmented_mrs_v3.xls')
augmrs_df = augmrs_df[augmrs_df['in_og'] == 0]
augmrs_df

,Unnamed: 0,conc_label_wx,conc_label_utf,conc_in_eng,mrs_conc,in_og
0,1,aBI_1,अभी_4,now_9,_now_a_1,0
1,2,PZilahAla_1,फ़िलहाल_1,now_10,_now_a_1,0
2,3,PilahAla_1,फिलहाल_1,now_11,_now_a_1,0
3,4,isa+samaya_1,इस+समय_1,now_12,_now_a_1,0
4,5,isa+vakwa_1,इस+वक्त_1,now_13,_now_a_1,0
...,...,...,...,...,...,...
43903,57164,bAriSa+ho_1,बारिश+हो_1,rain_25,_rain_v_1,0
43904,57165,barasAwa+ho_1,बरसात+हो_1,rain_26,_rain_v_1,0
43906,57167,pAnI+barasa_1,पानी+बरस_1,rain_28,_rain_v_1,0
43907,57168,meha+padZa_1,मेह+पड़_1,rain_29,_rain_v_1,0


In [219]:
def recount_word2(row):
    word = row['conc_label_utf'].split('_')[0]
    if word in mrs_word_cnt.keys():
        mrs_word_cnt[word] +=1
        return word+'_'+ str(mrs_word_cnt[word])
    else:
        mrs_word_cnt[word] =1
        return word+'_'+ str(mrs_word_cnt[word])
def recount_meaning2(row):
    meaning = row['conc_in_eng'].split('_')[0]
    if meaning in mrs_mean_cnt.keys():
        mrs_mean_cnt[meaning] += 1
        return meaning+'_'+ str(mrs_mean_cnt[meaning])
    else:
        mrs_mean_cnt[meaning] = 1
        return meaning+'_'+ str(mrs_mean_cnt[meaning])

In [220]:
augmrs_df['conc_label_utf'] = augmrs_df.apply(lambda row:recount_word2(row),axis =1)
augmrs_df['conc_in_eng'] = augmrs_df.apply(lambda row:recount_meaning2(row),axis =1)

C:\Users\aparashar\AppData\Local\Temp\ipykernel_20348\4222293487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  augmrs_df['conc_label_utf'] = augmrs_df.apply(lambda row:recount_word2(row),axis =1)
C:\Users\aparashar\AppData\Local\Temp\ipykernel_20348\4222293487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  augmrs_df['conc_in_eng'] = augmrs_df.apply(lambda row:recount_meaning2(row),axis =1)


In [458]:
augmrs_df

,conc_label_wx,conc_label_utf,conc_in_eng,mrs_conc
0,aBI_4,अभी_4,now_9,_now_a_1
1,PZilahAla_1,फ़िलहाल_1,now_10,_now_a_1
2,PilahAla_1,फिलहाल_1,now_11,_now_a_1
3,isa+samaya_1,इस+समय_1,now_12,_now_a_1
4,isa+vakwa_1,इस+वक्त_1,now_13,_now_a_1
...,...,...,...,...
41626,bAriSa+ho_1,बारिश+हो_1,rain_25,_rain_v_1
41627,barasAwa+ho_1,बरसात+हो_1,rain_26,_rain_v_1
41628,pAnI+barasa_1,पानी+बरस_1,rain_27,_rain_v_1
41629,meha+padZa_1,मेह+पड़_1,rain_28,_rain_v_1


In [223]:
mrs_word_cnt['अंक']
#mrs_mean_cnt['now']

18

In [224]:
augmrs_df['conc_label_wx'] = augmrs_df.conc_label_utf.apply(lambda x : con.convert(x))
augmrs_df

C:\Users\aparashar\AppData\Local\Temp\ipykernel_20348\241555038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  augmrs_df['conc_label_wx'] = augmrs_df.conc_label_utf.apply(lambda x : con.convert(x))


,Unnamed: 0,conc_label_wx,conc_label_utf,conc_in_eng,mrs_conc,in_og
0,1,aBI_4,अभी_4,now_9,_now_a_1,0
1,2,PZilahAla_1,फ़िलहाल_1,now_10,_now_a_1,0
2,3,PilahAla_1,फिलहाल_1,now_11,_now_a_1,0
3,4,isa+samaya_1,इस+समय_1,now_12,_now_a_1,0
4,5,isa+vakwa_1,इस+वक्त_1,now_13,_now_a_1,0
...,...,...,...,...,...,...
43903,57164,bAriSa+ho_1,बारिश+हो_1,rain_25,_rain_v_1,0
43904,57165,barasAwa+ho_1,बरसात+हो_1,rain_26,_rain_v_1,0
43906,57167,pAnI+barasa_1,पानी+बरस_1,rain_27,_rain_v_1,0
43907,57168,meha+padZa_1,मेह+पड़_1,rain_28,_rain_v_1,0


In [493]:
combined_df = pd.concat([ concat_df,augmrs_df])

In [255]:
#augmrs_df = augmrs_df.drop(['Unnamed: 0'],axis=1)
augmrs_df = augmrs_df.drop(['in_og'],axis=1)
#augmrs_df
#f_df
#combined_df

In [256]:
f_df.to_csv('mcgregor_concept_dict_v4.txt',sep='\t')
augmrs_df.to_csv('indo_wn_augm_concept_dict_v2.txt',sep='\t')
combined_df.to_csv('aggregate_concept_dict_v1.txt',sep='\t')

In [258]:
f_df.to_csv('mcgregor_concept_dict_v4.csv')
augmrs_df.to_csv('indo_wn_augm_concept_dict_v2.csv')
combined_df.to_csv('aggregate_concept_dict_v1.csv')

In [474]:
combined_df

,conc_label_wx,conc_label_utf,conc_in_eng,mrs_conc
0,A_1,आ_1,come_1,_come_v_1
1,Aba_1,आब_1,lustre_1,_lustre/NN_u_unknown
2,ABA_1,आभा_1,lustre_2,_lustre/NN_u_unknown
3,aba_1,अब_1,now_3,def_implicit_q
4,aba_1,अब_1,now_3,loc_nonsp
...,...,...,...,...
41626,bAriSa+ho_1,बारिश+हो_1,rain_25,_rain_v_1
41627,barasAwa+ho_1,बरसात+हो_1,rain_26,_rain_v_1
41628,pAnI+barasa_1,पानी+बरस_1,rain_27,_rain_v_1
41629,meha+padZa_1,मेह+पड़_1,rain_28,_rain_v_1


In [494]:
def rem_dgit(row):
    if row.split('_')[0].isdigit():
        return 1
    else:
        return 0

In [495]:
combined_df['isdigit'] = combined_df.conc_label_utf.apply(lambda x:rem_dgit(x))

In [496]:
combined_df = combined_df[combined_df['isdigit']==0]
combined_df.drop(['isdigit'],axis=1)

,conc_label_wx,conc_label_utf,conc_in_eng,mrs_conc
0,A_1,आ_1,come_1,_come_v_1
1,Aba_1,आब_1,lustre_1,_lustre/NN_u_unknown
2,ABA_1,आभा_1,lustre_2,_lustre/NN_u_unknown
3,aba_1,अब_1,now_3,def_implicit_q
4,aba_1,अब_1,now_3,loc_nonsp
...,...,...,...,...
41626,bAriSa+ho_1,बारिश+हो_1,rain_25,_rain_v_1
41627,barasAwa+ho_1,बरसात+हो_1,rain_26,_rain_v_1
41628,pAnI+barasa_1,पानी+बरस_1,rain_27,_rain_v_1
41629,meha+padZa_1,मेह+पड़_1,rain_28,_rain_v_1


In [497]:
combined_df['conc_label_utf'] = combined_df.conc_label_utf.apply(lambda x: x.strip('+'))

In [498]:
l = []
con = WXC(order='utf2wx')
def handle_6(row):
        if '+' in row['conc_label_utf']:
            try:
                split = row['conc_label_utf'].split('+')
                if '-' == split[0][-1]:
                        return '+'.join(split[1:])

                else:
                    return row['conc_label_utf']
            except:
                print(row['conc_label_utf'])

        else:
            return row['conc_label_utf']
    
def count_compound(row):
    if '+' in row['conc_label_utf'] and '_v_' not in row['mrs_conc']:
        ind = row['conc_label_utf'].split('_')[-1]
        split = row['conc_label_utf'].split('+')
        for i in range(len(split)-1):
            split[i] += '_'+ind
        return '+'.join(split)
    else:
        return row['conc_label_utf']
        
            
            
        

In [499]:
combined_df_2 = combined_df
combined_df_2['conc_label_utf'] = combined_df_2.apply(lambda row:handle_6(row),axis=1)
combined_df_2['conc_label_utf'] = combined_df_2.apply(lambda row:count_compound(row),axis=1)
combined_df_2['conc_label_wx'] = combined_df_2.conc_label_utf.apply(lambda x : con.convert(x))
combined_df_2 = combined_df_2.drop(['isdigit'],axis=1)

In [508]:
#combined_df_2 = combined_df_2.reset_index()
combined_df_2 = combined_df_2.drop(['index','level_0'],axis=1)

In [509]:
combined_df_2.to_csv('aggregate_concept_dict_v3.csv')
combined_df_2.to_csv('aggregate_concept_dict_v3.txt',sep='\t')

In [501]:

combined_df_dat = combined_df_2.assign(label='(concept_label-concept_in_Eng-MRS_concept')

combined_df_dat['mrs_conc'] = combined_df_dat.mrs_conc.apply(lambda x : x+')')
combined_df_dat=combined_df_dat.drop(['conc_label_utf'],axis=1)

In [502]:
cols = combined_df_dat.columns.tolist()
cols = ['label','conc_label_wx','conc_in_eng','mrs_conc']
combined_df_dat = combined_df_dat[cols]
combined_df_dat

,label,conc_label_wx,conc_in_eng,mrs_conc
0,(concept_label-concept_in_Eng-MRS_concept,A_1,come_1,_come_v_1)
1,(concept_label-concept_in_Eng-MRS_concept,Aba_1,lustre_1,_lustre/NN_u_unknown)
2,(concept_label-concept_in_Eng-MRS_concept,ABA_1,lustre_2,_lustre/NN_u_unknown)
3,(concept_label-concept_in_Eng-MRS_concept,aba_1,now_3,def_implicit_q)
4,(concept_label-concept_in_Eng-MRS_concept,aba_1,now_3,loc_nonsp)
...,...,...,...,...
41626,(concept_label-concept_in_Eng-MRS_concept,bAriSa+ho_1,rain_25,_rain_v_1)
41627,(concept_label-concept_in_Eng-MRS_concept,barasAwa+ho_1,rain_26,_rain_v_1)
41628,(concept_label-concept_in_Eng-MRS_concept,pAnI+barasa_1,rain_27,_rain_v_1)
41629,(concept_label-concept_in_Eng-MRS_concept,meha+padZa_1,rain_28,_rain_v_1)


In [503]:
combined_df_dat.to_csv('aggregate_concept_to_mrs_rels_v2.dat',sep=' ',index=False)

In [426]:
cft['_lustre/NN_u_unknown']

KeyError: '_lustre/NN_u_unknown'

In [425]:
def checj(row):
    if row not in cft.keys():
        print('aaaaaaaa')
combined_df.mrs_conc.apply(lambda x : checj(x))

aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
a

aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
a

aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
a

aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
a

0         None
1         None
2         None
3         None
4         None
          ... 
154191    None
154192    None
154193    None
154194    None
154195    None
Name: mrs_conc, Length: 154035, dtype: object